In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [3]:
df = fetch_hourly_rides(12)

2025-03-02 21:26:54,106 INFO: Initializing external client
2025-03-02 21:26:54,111 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:26:54,911 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.36s) 


In [4]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-02 11:00:00+00:00,195,2
1,2025-03-03 00:00:00+00:00,137,14
2,2025-03-03 00:00:00+00:00,131,0
3,2025-03-02 20:00:00+00:00,22,0
4,2025-03-02 18:00:00+00:00,46,0
...,...,...,...
4011,2025-03-02 13:00:00+00:00,158,52
4012,2025-03-02 10:00:00+00:00,177,2
4013,2025-03-03 00:00:00+00:00,91,0
4014,2025-03-02 16:00:00+00:00,185,0


In [5]:
df_pred = fetch_predictions(12)

2025-03-02 21:27:05,870 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 21:27:05,886 INFO: Initializing external client
2025-03-02 21:27:05,886 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:27:06,443 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 


In [6]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
246,192,0.0,2025-03-03 03:00:00+00:00
247,80,1.0,2025-03-03 03:00:00+00:00
248,215,1.0,2025-03-03 03:00:00+00:00
249,112,1.0,2025-03-03 03:00:00+00:00


In [7]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [8]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [9]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [10]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [11]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [12]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [13]:
mae_by_hour["MAE"].mean()

nan